In [28]:
%load_ext autoreload
%autoreload 2

#standard imports
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

#rl book imports
import rl
from rl.markov_decision_process import MarkovDecisionProcess
from rl.markov_process import State, MarkovProcess, NonTerminal, Terminal

from typing import (Callable, Dict, Generic, Iterator, Iterable, List,
                    Mapping, Optional, Sequence, Tuple, TypeVar, overload)


from rl.distribution import Categorical, Distribution, Constant, Choose
from rl.policy import Policy
from rl.monte_carlo import epsilon_greedy_policy, greedy_policy_from_qvf, glie_mc_control, mc_prediction
from rl.function_approx import LinearFunctionApprox, AdamGradient, Weights
from rl.td import glie_sarsa, q_learning
from rl.td_lambda import td_lambda_prediction


#custom imports 
import utils as u
import data as dat
import mdp_agent as ag
import baseline_policies as bp
import q_plots as qp
import backtest as btest
import v_predictor as v_true

import pickle 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [33]:
constant_params = {
    "mu":   lambda t : 10,
    "sigma": lambda t : 0.1,
    "kappa":  lambda t : 1/10
}

time_varying_params = {
    "mu":   lambda t : 100 + t*0.005,
    "kappa":   lambda t : 1/(15+5*np.cos(np.pi*t/30)),
    "sigma":    lambda t: 0.2    
}

params = constant_params

In [34]:
train, test = dat.build_simulated_train_test(N=10,mu=params["mu"],sigma=params["sigma"],kappa=params["kappa"])
u.plot_plotly_multiple(train)

# Load predictor

In [35]:
def intercept(x):
    return 1

def z_score_state(x):
    return (x.state["Spot"] - x.state["mu_t"])/(x.state["sigma_t"])
    

v_approx = LinearFunctionApprox.create(feature_functions=[intercept,z_score_state],weights=Weights(weights = np.array([-0.0017381 , -0.00776148]),
                                                                                        adam_cache1=0,adam_cache2=0,adam_gradient=0,time=0))

# Backtest

In [36]:
v_policy = bp.ActionFromV(v_approx,enter_long = 0.001,exit_long = 0.0005,enter_short=-0.001,exit_short= -0.0005)

trader = ag.Trading(train,test)
bt = btest.Backtester(trader, v_policy)
bt.summary()

/Users/mehdi/Desktop/cme241/RL_Project/utils.py:97: FutureWarning:

Argument `closed` is deprecated in favor of `inclusive`.

